# polygot fine tuning 데이터셋 제작

데이터 예시:

{"text": "### 명령어: 다음의 떡볶이에 대한 리뷰를 특성 세부설명 형식으로 요약하세요\n\n### 리뷰: 추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다\n\n### 요약: <양> 1패키지에 2 3인분 용량 <양> 1팩당 1165 칼로리 <재료> 밀떡 <조리> 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 <보관> 냉동보관<|endoftext|>"}

In [14]:
import pandas as pd
import re

data = pd.read_csv("/opt/ml/input/data/review_summary_prototype_dataset_v0.2_valid.csv")
data.set_index("id", inplace=True)
# data.iloc: 위치로 접근
# data.loc: id로 접근

# 특수문자, 초성어 제거
def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s0-9a-zA-Z]", " ", sent)
    sent_clean = re.sub("[ㄱ-ㅎㅏ-ㅣ]+", "", sent_clean)
    sent_clean = " ".join(sent_clean.split())
    return sent_clean

# data["input"] = data["input"].apply(clean_text)

display(data.tail(2))
print(data.iloc[0]["review"])

,Unnamed: 0,review_id,search_name,prod_name,review,summary
id,,,,,,
2,3,2,떡볶이,"떡볶이 추억의 국민학교 떡볶이 오리지널 (냉동), 600g, 2개",맛 가성비 재구매 의사 냉동 떡볶이 중 최고입니다. 떡볶이 당길 때 바로 해먹을 수...,<양> 세 식구 한끼 식사 <조리> 재료들을 한꺼번에 끓이면 돼서 조리가 편해요 <...
28,4,64,떡볶이,"떡볶이 오마뎅 진짜 부산 떡볶이 (냉동), 322g, 2개",제 다이어트 적 1순위는 단연코 떡볶이예요. 어쩜 먹어도 매일 맛있는지.. 덕분에 ...,<가격> 가격이 착하지는 않은데 <식감> 떡도 쫄깃한 식감에 어묵도 식감이 쫄깃하니...


매콤하고 달달하길 기대했는데 단맛이 더 강해요. 왜 이렇게 달콤하기만 하지 하며 먹었네요. 처음에 떡이 진짜 길이도 손가락만 하게 짧아서 양이 너무 적겠다. 싶었어요. 3개 들었다는 건 후기 보고 잘 알고 있었거든요. 그런데 다행히 요리하니 조금 퍼지며 먹기 좋은 양이 되더라고요. 신기신기 어묵이 특별히 더 맛있거나 쫄깃한 건 전혀 못 느꼈고요. 라면 사리는 넣길 잘했더라고요. 강추해요. 요. 떡볶이 한 세트로만은 양이 좀 적고 라볶이로 하니 한 끼 때우기 좋았네요.


## (성능 안 좋음) Tag 삭제

In [17]:
# import re

# def remove_tag(x):
#     xlist = [i.strip() for i in re.split("<[^>]*>", x) if len(i)>0]
#     return ". ".join(xlist)

# data["output"] = data["output"].apply(remove_tag)
# data["instruction"] = data["instruction"].apply(lambda x: re.sub(" <특성> 세부설명 형식으로", "", x) + ".")

# print(data.iloc[0]["instruction"], end="\n\n")
# print(data.iloc[0]["input"], end="\n\n")
# print(data.iloc[0]["output"], end="\n\n")

## 프롬프트 다른 버젼

```
### 상품: search_name + 상품명

### 리뷰:

### 요약:
```

In [18]:
# 0_e5: instruction에 <> 없음

## 입력 데이터 제작

In [19]:
def build_text(item):
    instruction = f"주어진 {item.search_name.strip()} 상품 리뷰에서 상품의 특성을 추출해서 요약하세요."
    
    text = ( # multi-line string
        "### 명령어: " + instruction + "\n\n"
        "### 리뷰: " + item.review.strip() + "\n\n"
        "### 요약: " + item.summary.strip() + "<|endoftext|>"
    )
    return text

print(build_text(data.iloc[0]))

### 명령어: 주어진 떡볶이 상품 리뷰에서 상품의 특성을 추출해서 요약하세요.

### 리뷰: 매콤하고 달달하길 기대했는데 단맛이 더 강해요. 왜 이렇게 달콤하기만 하지 하며 먹었네요. 처음에 떡이 진짜 길이도 손가락만 하게 짧아서 양이 너무 적겠다. 싶었어요. 3개 들었다는 건 후기 보고 잘 알고 있었거든요. 그런데 다행히 요리하니 조금 퍼지며 먹기 좋은 양이 되더라고요. 신기신기 어묵이 특별히 더 맛있거나 쫄깃한 건 전혀 못 느꼈고요. 라면 사리는 넣길 잘했더라고요. 강추해요. 요. 떡볶이 한 세트로만은 양이 좀 적고 라볶이로 하니 한 끼 때우기 좋았네요.

### 요약: <맛> 단맛이 더 강해요 <양> 떡이 퍼지며 먹기 좋은 양이 되더라구요 <맛> 오뎅이 특별히 더 맛있거나 쫄깃한 건 못 느꼈구요 <양> 떡볶이 한 세트로만은 양이 적고<|endoftext|>


In [20]:
data_list = []
for item in data.itertuples():
    text = build_text(item)
    item_dict = {
        "text": text
    }
    data_list.append(item_dict)
print(type(data_list[0]), data_list[0])

<class 'dict'> {'text': '### 명령어: 주어진 떡볶이 상품 리뷰에서 상품의 특성을 추출해서 요약하세요.\n\n### 리뷰: 매콤하고 달달하길 기대했는데 단맛이 더 강해요. 왜 이렇게 달콤하기만 하지 하며 먹었네요. 처음에 떡이 진짜 길이도 손가락만 하게 짧아서 양이 너무 적겠다. 싶었어요. 3개 들었다는 건 후기 보고 잘 알고 있었거든요. 그런데 다행히 요리하니 조금 퍼지며 먹기 좋은 양이 되더라고요. 신기신기 어묵이 특별히 더 맛있거나 쫄깃한 건 전혀 못 느꼈고요. 라면 사리는 넣길 잘했더라고요. 강추해요. 요. 떡볶이 한 세트로만은 양이 좀 적고 라볶이로 하니 한 끼 때우기 좋았네요.\n\n### 요약: <맛> 단맛이 더 강해요 <양> 떡이 퍼지며 먹기 좋은 양이 되더라구요 <맛> 오뎅이 특별히 더 맛있거나 쫄깃한 건 못 느꼈구요 <양> 떡볶이 한 세트로만은 양이 적고<|endoftext|>'}


In [21]:
import json
with open("../../data/review_summary_prototype_textonly_v0.2_valid.json", "w", encoding="utf-8") as f:
    json.dump(data_list, f, ensure_ascii=False)

# v1.0 프롬프트

In [5]:
# json에서 가져오기
import pandas as pd
import json

# data = pd.read_json("/opt/ml/input/data/review_summary_v1.0.json")
data = pd.read_json("/opt/ml/input/data/v3.5/summary_v3.5_1500_train.json")
data = data[["id", "prod_name", "review", "summary"]]

def build_text(item):

    text = (  # multi-line string
        f"### 명령어: 다음 상품에 대한 리뷰에서 상품의 특성을 요약하세요\n\n"
        "### 리뷰: " + item.review + "\n\n"
        "### 요약: " + item.summary + "<|endoftext|>"
    )
    return text

data_list = []
for item in data.itertuples():
    text = build_text(item)
    item_dict = {
        "text": text
    }
    data_list.append(item_dict)
    
with open("/opt/ml/input/data/v3.5/koalpaca_textonly.json", "w", encoding="utf-8") as f:
    json.dump(data_list, f, ensure_ascii=False)

## 데이터셋 잘 불러오는지 확인

In [6]:
from datasets import load_dataset
dataset = load_dataset(
            "json",
            data_files="/opt/ml/input/data/v3.5/koalpaca_textonly.json",
            # cache_dir=model_args.cache_dir,
            # use_auth_token=True if model_args.use_auth_token else None,
            # **dataset_args,
        )

/opt/conda/envs/common/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 936.86it/s]


Dataset json downloaded and prepared to /opt/ml/.cache/huggingface/datasets/json/default-986d325c93c547a5/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 557.38it/s]


In [7]:
print(dataset)
print(dataset["train"]["text"][0])

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6033
    })
})
### 명령어: 다음 상품에 대한 리뷰에서 상품의 특성을 요약하세요

### 리뷰: 여러 번 씻고 밥을 하니 밥하는 냄새부터 아주 고소합니다. 다 된 밥 열어보니 역시 믿고 먹는 메뚜기쌀이네. 요 윤기도 좔좔 찰기도 쫀득 너무 맛있네요. 밥만 먹어도 달달함과 고소함이 있어요. 첫 수저를 먹자마자 어 이게 아닌데 쓰 읍했습니다. 제 메모장에 적어 놓았지요 쌀 포대 풀 때 정말 편리해요. 판매자님의 배려를 200 느낍니다. 앞으로도 변치 않는 품질로 사업하시어 대박 민소매기를 바랍니다. 쌀 10kg 이면 15리터 페트병으로 5개 몇 컵 정도 나와요. 거의 저녁만 먹는 유아 2명 4인 가족이 한 달 정도 먹는 양이에요. 쌀눈이 꽤 많이 붙어 있어요. 밥을 하면 적당히 찰지고 적당히 윤기가 나는 밥이 됩니다. 등급 표시를 보니 이렇게 되어있네요. 쌀눈은 약간 반투명색이고 쌀 크기는 크지도 작지도 않네요. 저희 집은 그냥 흰밥을 먹지 않고 흑미 한주먹씩 섞어서 같이 먹는답니다. 근데 김밥 쌀 때우는 그냥 흰쌀만 해도 밥맛이 좋더라고요. 메뚜기쌀은 굳이 불리지 않고 밥을 바로 해도 고소하고 윤기 있는 맛있는 밥이 돼 더라고요. 가성비도 좋고 국산 창녕 쌀에 맛도 무난해서 전 다 먹고 나서도 꾸준히 메뚜기쌀 또 먹을 거예요. 2020년 생산된 쌀로 12월 10일 도정된 최근 일자 백이랍니다. 굿네이버스에 착한 선행에도 앞장선다고 하니 왠지 더더욱 믿음도 가고 자주 애용도 해야 할 것 같다는 생각이 들었습니다. 물에 담가 씻어보니 생각보다 쌀뜨물이 뿌옇습니다. 현미찹쌀을 조금 섞어서 지어 먹었는데 향도 좋고 맛도 좋네요. 윤기가 좔좔 흐르는 쌀밥은 안되지만 가성비 좋은 쌀이므로 만족하고 먹겠습니다. 농협이라고 마크 붙어 있는 것이 항상 쌀알도 매끈하고 도정이 깨끗하게 와요. 마트 쌀은 20킬로는 5만 6만

# KoAlpaca 추가 fine tuning 데이터셋 제작

KoAlpaca 예시:

{"text":"### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>"}

In [7]:
import pandas as pd
import re

data = pd.read_csv("../../data/review_summary_prototype_dataset.csv")
data.set_index("id", inplace=True)
# data.iloc: 위치로 접근
# data.loc: id로 접근

# 특수문자, 초성어 제거
def clean_text(sent)->str:
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s0-9a-zA-Z]", " ", sent)
    sent_clean = re.sub("[ㄱ-ㅎㅏ-ㅣ]+", "", sent_clean)
    sent_clean = " ".join(sent_clean.split())
    return sent_clean

data["input"] = data["input"].apply(clean_text)

data.head(2)

,search_name,instruction,input,output
id,,,,
707,떡볶이,다음의 떡볶이에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요,추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 2...,<양> 1패키지에 2 3인분 용량 <양> 1팩당 1165 칼로리 <재료> 밀떡 <조...
708,떡볶이,다음의 떡볶이에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요,맛 가성비 재구매의사 냉동 떡볶이중 최고입니다떡복이 땡길때 바로 해먹을수 있어서 좋...,<양> 세 식구 한끼 식사 <조리> 재료들을 한꺼번에 끓이면 돼서 조리가 편해요 <...


In [8]:
def build_text(item):
    question = re.sub("[<>]", "",item.instruction.strip()) # 0_e5와 같은 형식
    
    text = ( # multi-line string
        "### 질문: " + question + ".\n\n"
        "" + clean_text(item.input.strip()) + "\n\n"
        "### 답변: " + item.output.strip() + "<|endoftext|>"
    )
    return text

print(build_text(data.iloc[0]))

### 질문: 다음의 떡볶이에 대한 리뷰를 특성 세부설명 형식으로 요약하세요.

추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다

### 답변: <양> 1패키지에 2 3인분 용량 <양> 1팩당 1165 칼로리 <재료> 밀떡 <조리> 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 <보관> 냉동보관<|endoftext|>


In [9]:
data_list = []
for item in data.itertuples():
    text = build_text(item)
    item_dict = {
        "text": text
    }
    data_list.append(item_dict)
print(type(data_list[0]), data_list[0])

<class 'dict'> {'text': '### 질문: 다음의 떡볶이에 대한 리뷰를 특성 세부설명 형식으로 요약하세요.\n\n추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다\n\n### 답변: <양> 1패키지에 2 3인분 용량 <양> 1팩당 1165 칼로리 <재료> 밀떡 <조리> 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 <보관> 냉동보관<|endoftext|>'}


In [10]:
import json
with open("../../data/review_summary_prototype_textonly_koalpaca.json", "w", encoding="utf-8") as f:
    json.dump(data_list, f, ensure_ascii=False)

: 

# 데이터 토큰 개수 확인

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")

In [8]:
print(tokenizer("안녕하세요"))
tokenizer.tokenize("안녕하세요")

{'input_ids': [13417, 284, 1934], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


['ìķĪëħķ', 'íķĺ', 'ìĦ¸ìļĶ']

In [15]:
import json
import pandas as pd
with open("../data/review_summary_prototype_textonly.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)
df = pd.DataFrame(data_list)
df.sample(1)


,text
17,### 명령어: 다음의 떡볶이에 대한 리뷰를 특성 세부설명 형식으로 요약하세요\n\...


In [16]:
def find_length(text):
    ids = tokenizer(text).input_ids
    return len(ids)

df["length"] = df["text"].apply(lambda x: find_length(x))
df.sample(1)

,text,length
38,### 명령어: 다음의 떡볶이에 대한 리뷰를 특성 세부설명 형식으로 요약하세요\n\...,297


In [17]:
df.describe()

,length
count,87.000000
mean,272.781609
std,66.955630
min,79.000000
25%,235.000000
50%,275.000000
75%,313.500000
max,399.000000


# 평가 데이터 분리

In [11]:
import pandas as pd
import re

df = pd.read_csv("/opt/ml/input/data/review_summary_prototype_dataset_v0.2.csv")
df = df.sample(frac=1).reset_index(drop=True)
valid = df.iloc[:5]
train = df.iloc[5:]
valid.to_csv("/opt/ml/input/data/review_summary_prototype_dataset_v0.2_valid.csv")
train.to_csv("/opt/ml/input/data/review_summary_prototype_dataset_v0.2_train.csv")

# 추론

In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM
# MODEL = "models/0_e5" # 가장 결과 좋음
# MODEL = "models/koalpaca-ft-myFormat"
MODEL = "models/koalpaca-ft"
# MODEL = "models/0_noTag"
# MODEL = "models/e10"
# MODEL = "models/e5lr1/"
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline(
    'text-generation', 
    model=model,
    tokenizer=MODEL,
    device=0
)

/opt/conda/envs/mrc/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.08s/it]
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [2]:
def ask(keyword, review, is_input_full=False):
    text = ( # multi-line string
        "### 질문: 다음의 {keyword}에 대한 리뷰를 특성 세부설명 형식으로 요약하세요.\n\n"
        "" + review + "\n\n"
        "### 답변:"
    )
    
    ans = pipe(
        text, 
        do_sample=True, 
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    result = ans[0]['generated_text']
    return result

# def ask(keyword, review, is_input_full=False):
#     ans = pipe(
#         f"### 명령어: 다음의 {keyword.strip()}에 대한 리뷰를 특성 세부설명 형식으로 요약하세요\n\n### 리뷰: {review}\n\n### 요약:", 
#         do_sample=True, 
#         max_new_tokens=512,
#         temperature=0.7,
#         top_p=0.9,
#         return_full_text=False,
#         eos_token_id=2,
#     )
#     result = ans[0]['generated_text']
#     return result

In [9]:
test_data = [
    ("떡볶이", "일단 결론부터 말씀드리자면 이 떡볶이 진짜 맛있어요진짜 맛은 있는데 생각보다 양이 너무 적어서 ㅠㅠ 그게 좀 아쉽솔직히 혼자서도 충분히 다 먹을 수 있을 양이고요 그냥 딱 1인분인듯 ㅋㅋ 그래도 그나마 2봉지라 다행인건지 ㅠㅠ저는 사실 이 떡볶이만 벌써 4번째 재구매고요2봉지가 한세트 구성인데 저는 한 번 구매할때마다 2세트씩 구매하고 있답니다그만큼 맛에서는 충분히 만족하고 있는 떡볶이이긴 하나 가격대비 양이 너무 적은 건 좀 섭섭함 ㅋㅋㅋ그리고 저는 사실 떡보다는 오뎅을 더 좋아하는 편이라오뎅을 따로 더 넣어서 조리하는 편입니다집에 양배추랑 대파도 있으시면 왠만하면 넣으세용 저는 오뎅 양배추 대파 삶은계란까지 꼭꼭 넣어서 먹거든요 그렇게 먹으니 진짜 맛있더라고요 ㅋㅋㅋ아 그리고 사리종류로 라면사리나 당면사리를 살짝 넣어드셔도 좋아요저는 라면사리보다는 당면사리를 좀 더 좋아하긴 합니다 키키키키아무튼간에 이 떡볶이 안드셔보신 분이 계시다면강력하게 추천드립니다 ㅋㅋㅋ 꼭꼭 드셔보세용"),
    ("떡볶이", "간단한 간식거리를 찾다 보니 두께라는 이름에 한 떡볶이집이 생각나서 홀린 듯이 구입하게 되었네요구성은 여느 간편조리 떡볶이와 비슷한데 왠지 모르게 별거 아닌데도 맘에 드는 모양새였어요 만드는 법도 간단하고 만들고 나서의 모습도 완벽한 이상적인 떡볶이의 형태여서 맛있게 잘 먹었습니다"),
    ("떡볶이", "떡볶이 처돌이라 여러종류 시켜먹어보는데 무난한 맛입니다 단맛이 살짝 아쉬워서 양파나 양배추 넣어서 먹으면 좋을 것 같습니다 저같은 경우는 치즈한 장 넣어서 떡볶이랑 먹으니 정말 맛있었습니다 떡볶이 떡이 길어서 뭔가 먹는 재미가 있었습니다"),
    ("떡볶이", "아무리 생각해도 이상해요떡볶이는 왜 먹어도 먹어도 질리지 않을까요모든 떡볶이를 다 좋아하는데 요즘은 이 상품에 꽂혔어요미미네 떡볶이미미네 한창 유행할 때 가게에서 먹어보고다시 먹으러 찾아가기는 귀찮아서 혹시나하고 쿠팡에서 검색해봤어요역시나 로켓 프레시 배송이 되서 급 기분 좋아지더라고요다음 날 새벽 배송 바로 받고 점심으로 보글보글 끓여 먹어봤어요음 역시나 조리법도 너무 간단하고 맛도 평타 이상이예요아무래도 떡이 냉동되서 끓이면 갈라지고 냉장떡보다는 좀 풀어진다고해야하나 그렇기는 합니다그래도 떡볶이는 양념 맛이 중요하니까 아주 먹을만하고 만족스러워요다른거보다 국물 떡볶이라서 떠먹는 국물이 맛이 좋아서 합격 그리고 어묵이 포함이 되서 더 괜찮은 거 같네요추천합니다"),
    ("치킨", "평소에 가라아게 치킨좋아해서 보이면 사는편입니다물가올라서 1k 비싼편인데 착한가격이라 사봄 500g 23인분다리살 부드러움국내산이 아니라아쉽지만브라질산잡내없이 맛있음 에어프라이 180 15분 기본으로 굽고 좀더 바싹함을 원함 5분정도 더 구움되요기본만 구움 좀 물컹하더라구요 갠적으로 바싹한 느낌을좋아해서요 기본적으로 간이 되어있어서 맛은 좋아요양은 한봉다리 다 뜯은거구요 엄청 많지는않지만 가격대비는괜찮은듯 해요더 짭짭한거 좋아하시는분들은 소스 찍어드셔도좋을것 같아요"),
    ("치킨", "유명한 애 보다 고기 함량도 10프로 높아요덜 짜고 육즙도 한가득이엇어요에어프라이어 14분정도 돌렸더니바삭하고 맛잇어요전에 주문해서 먹고 남았던 치킨소스남은거랑 머스타드 소스 같이 하니굳 이네요 마지막에 파슬리를솔솔 뿌려 주었더니 더 보기 좋네요맥주 안주 겸 아이 간식 겸 주문 했는데 너무 만족스럽습니다이틀만에 다먹어버렸어요 ㅠㅠㅠㅠ완전 추천드립니다"),
    ("치킨", "우연히 배달음식 사이드로 먹어본 가라아게가너무 맛있어서 찾고 있는데 이건 아니었어요ㅠ사세 다른 시리즈 가라아게도 도전해보려고요"),
    ("치킨", "내돈내산 구매 후기 입니다주관적일 수 밖에 없지만 최대한 객관적으로 적어보겠습니다상품명 사세 치킨 가라아게 냉동 500g 2개구매일시 23 05 12도착일시 23 05 13재구매 의사 있음사세 핫윙인가 여튼 그 닭날개가 유명한데다른 제품들도 다 기본 이상으로 맛있어요 그래도 이 가라아게는 처음 먹어보는데저는 일단 가라아게를 무척이나 좋아하는 편이 가라아게 맛이 좋은 편이에요다른 싸구려 가라아게 보다 훨씬 고급진 느낌겉은 바삭하면서속이 좀 부드러운 것보다 쫀득한 느낌 기름에 튀겨도 맛있고그냥 기름 살짝 버무려서 에어프라이에 돌려도아주 맛이 좋습니다 만족 조금이라도 도움이 되셨다면 아래 도움이 돼요버튼 한번 눌러 주시면 감사하겠습니다")   
]

In [10]:
import re

# 특수문자, 초성어 제거
def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s0-9a-zA-Z]", " ", sent)
    sent_clean = re.sub("[ㄱ-ㅎㅏ-ㅣ]+", "", sent_clean)
    sent_clean = " ".join(sent_clean.split())
    return sent_clean

In [11]:
import json
import os

summary = []
for keyword, review in test_data:
    clean_review = clean_text(review)
    result = ask(keyword, clean_review)
    item = {"review": clean_review, "summary": result}
    summary.append(item)

name = MODEL.strip().split(".")[0].split("/")[-1]
path = "test_result/"+name + ".json"

os.makedirs("test_result", exist_ok=True)

with open(path, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False)

print("file:", path)

# for item in summary:
#     print("-"*5, "review", "-"*5)
#     print(item["review"])
#     print("-"*5, "summary", "-"*5)
#     print(item["summary"])  


/opt/conda/envs/mrc/lib/python3.8/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


file: test_result/koalpaca-ft.json


# 평가

In [23]:
import evaluate
rouge = evaluate.load("rouge")

In [24]:
answer = [
    "<가격> 착한가격 <재료> 다리살 부드러움 <원산지> 브라질산 <맛> 잡내없이 맛있음 간이 되어있어서 맛은 좋아요 <조리> 에어프라이 <양> 엄청 많지는않지만 가격대비는괜찮은듯",
    "<맛> 덜 짜고 육즙도 한가득 <재료> 유명한 애 보다 고기 함량도 10프로 높아요 <조리> 에어프라이어 <소스> 킨소스남은거 머스타드 소스 <선호> 맥주 안주 아이 간식",
    "<맛> 이건 아니었어요",
    "<맛> 가라아게 맛이 좋은 편 훨씬 고급진 느낌 <식감> 겉은 바삭하면서속이 좀 부드러운 것보다 쫀득한 느낌 <조리> 기름에 튀겨 기름 살짝 버무려서 에어프라이에 돌려"
]

In [25]:
import json
with open("test_result/0_e5.json", "r") as f:
    preds = json.load(f)
preds = [p["summary"].strip() for p in preds[-4:]]
print(preds)

['<가격> 착한가격 <재료> 다리살 부드러움 <원산지> 브라질산 <맛> 잡내없이 맛있음 간이 되어있어서 맛은 좋아요 <조리> 에어프라이 <양> 엄청 많지는않지만 가격대비는괜찮은듯', '<맛> 덜 짜고 육즙도 한가득 <식감> 바삭하고 맛잇어요 <재료> 유명한 애 보다 고기 함량도 10프로 높아요 <양> 에어프라이어 14분정도 돌렸더니 <조리> 바삭하고 맛잇어요 <소스> 남은거랑 머스타드 소스', '<맛> 짭짤하고 바삭바삭 <식감> 튀김옷이 두껍지 않아 얇아 <재료> 닭다리살 <조리> 에어프라이기 <양> 양도 많아 <가격> 가성비 좋은', '<맛> 기본 이상으로 맛있다 <재료> 닭날개가 유명한데 <조리> 에어프라이 <식감> 겉은 바삭하면서 속은 쫀득한 느낌 <재료> 가라아게를 무척이나 좋아하는 편']


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("models/0_e5/")

In [17]:
from pprint import pprint
result = rouge.compute(predictions=preds, 
              references=answer,      
              tokenizer=tokenizer.tokenize,
              use_aggregator=False)
result_agg = rouge.compute(predictions=preds, 
              references=answer,      
              tokenizer=tokenizer.tokenize,
              use_aggregator=True) # default
pprint(result)
pprint(result_agg)

{'rouge1': [1.0, 0.736842105263158, 0.1111111111111111, 0.5454545454545455],
 'rouge2': [1.0, 0.6428571428571428, 0.07692307692307691, 0.43298969072164945],
 'rougeL': [1.0, 0.6842105263157895, 0.1111111111111111, 0.3838383838383838],
 'rougeLsum': [1.0, 0.6842105263157895, 0.1111111111111111, 0.3838383838383838]}
{'rouge1': 0.5983519404572036,
 'rouge2': 0.5381924776254673,
 'rougeL': 0.544790005316321,
 'rougeLsum': 0.544790005316321}


# 모델 허깅페이스 업로드

Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL = "/opt/ml/input/KoAlpaca-finetune/train_v1.1b/models/v0.2"
REPO = "boostcamp-5th-nlp07/koalpaca-polyglot-5.8b-summary-v0.2"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)

model.push_to_hub(REPO)
tokenizer.push_to_hub(REPO)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.87G [00:00<?, ?B/s]